In [1]:
!pip install polars==0.20.4 #Colab uses the older version and it doesnt support some features
import os
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from typing import List
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
try:
    import optuna
except:
    !pip install optuna
    import optuna

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c predict-energy-behavior-of-prosumers #Download the dataset
!unzip predict-energy-behavior-of-prosumers #Unzip the dataset

 96% 224M/233M [00:01<00:00, 171MB/s]
100% 233M/233M [00:01<00:00, 141MB/s]
Archive:  predict-energy-behavior-of-prosumers.zip
  inflating: client.csv              
  inflating: county_id_to_name_map.json  
  inflating: electricity_prices.csv  
  inflating: enefit/__init__.py      
  inflating: enefit/competition.cpython-310-x86_64-linux-gnu.so  
  inflating: example_test_files/client.csv  
  inflating: example_test_files/electricity_prices.csv  
  inflating: example_test_files/forecast_weather.csv  
  inflating: example_test_files/gas_prices.csv  
  inflating: example_test_files/historical_weather.csv  
  inflating: example_test_files/revealed_targets.csv  
  inflating: example_test_files/sample_submission.csv  
  inflating: example_test_files/test.csv  
  inflating: forecast_weather.csv    
  inflating: gas_prices.csv          
  inflating: historical_weather.csv  
  inflating: public_timeseries_testing_util.py  
  inflating: train.csv               
  inflating: weather_station_to_c

In [4]:
colab_path = Path("/content")
train = pl.read_csv(os.path.join(colab_path, "train.csv"), try_parse_dates = True)

**DATA ANALYSIS**

In [5]:
pd_train = pd.read_csv(os.path.join(colab_path, "train.csv"))
pd_train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [6]:
pd_train.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id'],
      dtype='object')

In [ ]:
import seaborn as sns
plt.figure(figsize = (10, 6))

for feature in pd_train.columns:
    sns.barplot(data = pd_train[:10], x = feature, y = "target")
    plt.show()

**FEATURE ENGINEERING**

In [8]:
def feature_engineering(df_data, df_client, df_forecast, df_historical, df_location, df_target):
    df_data = df_data.with_columns(pl.col("datetime").cast(pl.Date).alias("date"))
    df_client = df_client.with_columns(
        (pl.col("date") + pl.duration(days = 2)).cast(pl.Date)
    )
    df_location = df_location.with_columns(
        pl.col("latitude").cast(pl.datatypes.Float32),
        pl.col("longitude").cast(pl.datatypes.Float32))
    df_forecast = df_forecast.rename({"forecast_datetime": "datetime"}).with_columns(
        pl.col("latitude").cast(pl.datatypes.Float32),
        pl.col("longitude").cast(pl.datatypes.Float32)
    ).join(df_location, how = "left", on = ["longitude", "latitude"]).drop("longitude", "latitude")
    df_historical = df_historical.with_columns(
        pl.col("latitude").cast(pl.datatypes.Float32),
        pl.col("longitude").cast(pl.datatypes.Float32,),
        pl.col("datetime") + pl.duration(hours = 37)
    ).join(df_location, how = "left", on = ["longitude", "latitude"]).drop("longitude", "latitude")
    df_forecast_date = df_forecast.group_by("datetime").mean().drop("county")
    df_forecast_local = df_forecast.filter(pl.col("county").is_not_null()).group_by("county", "datetime").mean()
    df_historical_date = df_historical.group_by("datetime").mean().drop("county")
    df_historical_local = df_historical.filter(pl.col("county").is_not_null()).group_by("county", "datetime").mean()

    df_data = (df_data
        .join(df_client, on = ["county", "is_business", "product_type", "date"], how = "left")
        .join(df_forecast_date, on = "datetime", how = "left", suffix = "_fd")
        .join(df_forecast_local, on = ["county", "datetime"], how = "left", suffix = "_fl")
        .join(df_historical_date, on = "datetime", how = "left", suffix = "_hd")
        .join(df_historical_local, on = ["county", "datetime"], how = "left", suffix = "_hl")
        .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on = "datetime", how = "left", suffix = "_fdw")
        .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on = ["county", "datetime"], how = "left", suffix = "_flw")
        .join(df_historical_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on = "datetime", how = "left", suffix = "_hdw")
        .join(df_historical_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on = ["county", "datetime"], how = "left", suffix = "_hlw")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on = ["county", "is_business", "product_type", "is_consumption", "datetime"], how = "left")
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
            pl.col("datetime").dt.hour().alias("hour"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.year().alias("year")
        )
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator = "_").alias("category_1"),
        )
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)")
        )
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
        .drop("date", "datetime", "hour", "dayofyear")
    )

    return df_data

In [31]:
def to_pandas(x, y = None):
    COL_S = ["county", "is_business", "product_type", "is_consumption", "category_1"]

    if y is not None:
        df = pd.concat([x.to_pandas(), y.to_pandas()], axis = 1)
    else:
        df = x.to_pandas()

    df = df.set_index("row_id") #Set the column "row_id" as the index/row of the dataframe
    df[COL_S] = df[COL_S].astype("category") #Change the COL_S type to "category"

    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)

    return df

In [10]:
def objective(trial):
    #Use ml_model = trial.suggest_categorical("") to try other ml methods
    params = {
        "objective": "regression",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 10.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 4, 256),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "max_bin": trial.suggest_int("max_bin", 32, 1024)
    }

    model = lgb.LGBMRegressor(**params)

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    error = mean_absolute_error(y_pred, y_test)
    return error

In [11]:
data_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id']
client_cols = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
forecast_cols = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols = ['longitude', 'latitude', 'county']
target_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

In [12]:
df_data = pl.read_csv(os.path.join(colab_path, "train.csv"), columns = data_cols, try_parse_dates = True)
df_client = pl.read_csv(os.path.join(colab_path, "client.csv"), columns = client_cols, try_parse_dates = True)
df_forecast = pl.read_csv(os.path.join(colab_path, "forecast_weather.csv"), columns = forecast_cols, try_parse_dates = True)
df_historical = pl.read_csv(os.path.join(colab_path, "historical_weather.csv"), columns = historical_cols, try_parse_dates = True)
df_location = pl.read_csv(os.path.join(colab_path, "weather_station_to_county_mapping.csv"), columns = location_cols, try_parse_dates = True)
df_target = df_data.select(target_cols)

In [20]:
x, y = df_data.drop("target"), df_data.select("target")
x = feature_engineering(x, df_client, df_forecast, df_historical, df_location, df_target)

In [32]:
df_train = to_pandas(x, y)

In [33]:
df_train = df_train[df_train["target"].notnull() & df_train["year"].gt(2021)]

In [34]:
x_train, x_test, y_train, y_test = train_test_split(df_train.drop("target", axis = 1), df_train["target"], test_size = 0.4, shuffle = True)

In [17]:
study = optuna.create_study()
optimized_study = study.optimize(objective, n_trials = 50, show_progress_bar = True)

In [25]:
best_params_lgb_d = {
    'learning_rate': 0.05689066836106983,
    'lambda_l1': 3.6277555139102864,
    'lambda_l2': 1.6591278779517808,
    'min_data_in_leaf': 186,
    'max_depth': 9,
    'max_bin': 813,
}

In [35]:
from joblib import dump, load
voting_reg = load("/content/drive/MyDrive/SavedModels/voting_regressor_xgb.joblib")

In [30]:
x_test.head()

,county,is_business,product_type,is_consumption,eic_count,installed_capacity,hours_ahead,temperature,dewpoint,cloudcover_high,...,target_7,day,weekday,month,year,category_1,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour)
row_id,,,,,,,,,,,,,,,,,,,,,
1343787,11,1,2,1,7.0,119.000000,24.0,8.739361,6.791767,0.546306,...,40.617001,2,3,11,2022,11_1_2_1,-0.857315,0.514793,-2.588190e-01,-0.965926
1022405,3,0,1,1,30.0,311.000000,24.0,21.654451,14.479589,0.684719,...,8.431000,25,1,7,2022,3_0_1_1,-0.384665,-0.923056,-5.000000e-01,-0.866025
568506,11,1,3,0,119.0,5794.899902,16.0,-1.336159,-2.924360,0.380288,...,0.003000,6,7,3,2022,11_1_3_0,0.898292,0.439400,9.659258e-01,0.258819
1363506,10,1,2,0,5.0,41.000000,23.0,9.244884,7.414896,0.009429,...,0.129000,8,2,11,2022,10_1_2_0,-0.799839,0.600214,1.224647e-16,-1.000000
1031180,11,0,2,0,7.0,14.500000,18.0,15.905991,12.133893,0.411054,...,0.440000,28,4,7,2022,11_0_2_0,-0.431673,-0.902030,8.660254e-01,-0.500000


In [36]:
votinh_reg_xgb_pred = voting_reg.predict(x_test)

In [37]:
r2_score(votinh_reg_xgb_pred, y_test)

0.9905897755332761

In [ ]:
model = VotingRegressor([
    ("lgb_1", lgb.LGBMRegressor(**best_params_lgb_d, random_state = 100)),
    ("lgb_2", lgb.LGBMRegressor(**best_params_lgb_d, random_state = 101)),
    ("lgb_3", lgb.LGBMRegressor(**best_params_lgb_d, random_state = 102)),
])

model.fit(x_train, y_train)

In [27]:
voting_regressor_pred = model.predict(x_test)

[LightGBM] [Warning] min_data_in_leaf is set=186, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=186
[LightGBM] [Warning] lambda_l2 is set=1.6591278779517808, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6591278779517808
[LightGBM] [Warning] lambda_l1 is set=3.6277555139102864, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.6277555139102864
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] min_data_in_leaf is set=186, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=186
[LightGBM] [Warning] lambda_l2 is set=1.6591278779517808, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6591278779517808
[LightGBM] [Warning] lambda_l1 is set=3.6277555139102864, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.6277555139102864
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2

In [ ]:
plt.figure(figsize = (10, 6))
plt.scatter(y_test[:4000], voting_regressor_pred[:4000], alpha = 0.5)
plt.xlabel("Asıl Değer")
plt.ylabel("Tahmin Edilen Değer")
plt.title("Asıl vs Tahmin Edilen Değer")
plt.plot([y_test[:4000].min(), y_test[:4000].max()], [y_test[:4000].min(), y_test[:4000].max()])
plt.show()

In [23]:
r2_score(voting_regressor_pred, y_test)

0.9762176170192084

In [ ]:
xgb_model = VotingRegressor([
    ("xgb_1", XGBRegressor(**best_params_lgb_d, enable_categorical = True)),
    ("xgb_2", XGBRegressor(**best_params_lgb_d, enable_categorical = True)),
    ("xgb_3", XGBRegressor(**best_params_lgb_d, enable_categorical = True)),
])

xgb_model.fit(x_train, y_train)

In [ ]:
y_pred = xgb_model.predict(x_test)

In [ ]:
r2_score(np.array(y_pred), y_test)

In [ ]:
dump(xgb_model, "/content/drive/MyDrive/SavedModels/voting_regressor_xgb.joblib")
#voting_reg = load("voting_regressor_xgb.joblib") if you want to load the model

dump(model, "/content/drive/MyDrive/SavedModels/voting_regressor_lgb.joblib")
#voting_reg = load("voting_regressor_lgb.joblib") if you want to load the model